In [1]:
import seaborn as sns
import os, sys
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import pandas as pd 

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import label_binarize


from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input as preprocess_input_vgg19
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input as preprocess_input_resnet50

from keras.layers.pooling import GlobalAveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers import Input, Dense
from keras.layers.core import Dropout, Activation
from keras.callbacks import ModelCheckpoint
from keras.layers.normalization import BatchNormalization
from keras.models import Model

import datetime
from keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.metrics import accuracy_score
from keras.preprocessing.image import ImageDataGenerator


from sklearn.utils import shuffle

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
from os import listdir, makedirs
from os.path import join, exists, expanduser

from keras.applications.xception import Xception
from keras.applications.xception import preprocess_input as preprocess_input_Xception
from keras.models import load_model

import cv2

Using TensorFlow backend.


# Détection automatique d'angle (brouillon)

Ce notebook implémente une ébauche d'algorithme de détection automatique d'angles entre -pi/5 et +pi/5. On sélectionne le meilleur hyperparamètre
(ici dropout pour la régularisation, taille des mini-batchs et nombre de neurones sur les couches ajoutées) sur le jeu de validation en visualisant l'évolution de la fonction de cout. On quantifie la performances via une RMSE sur jeu de test. On utilise ensuite les angles détectés pour créer des images dont le cadre est incliné de l'opposé de cet angle. Le contenu de l'image est, quand à lui, devenu droit. On utilise une fonction d'activation softmax sur le dernier neurone représentant l'angle redimmensionné entre 0 et 1 (0 correspondant à un angle de -pi/5 rad et 1, +pi/5 rad)


In [1]:
import tensorflow as tf
tf.__version__

'1.13.1'

In [3]:
dim_Xception = 299

# ouvrir le fichier 

labels = pd.read_csv('liste_label.csv')

In [4]:
X_train = np.zeros((nb_train,dim_Xception,dim_Xception,3),dtype=np.float32)

y_train = np.zeros(nb_train)

X_test = np.zeros((nb_test,dim_Xception,ddim_Xception,3),dtype=np.float32)

y_test = np.zeros(nb_test)


X_valid = np.zeros((nb_valid,dim_Xception,dim_Xception,3),dtype=np.float32)

y_valid = np.zeros(nb_valid)

In [5]:
j_valid = 0

# Open a file
path = "./images_t/valid/"
dirs = os.listdir( path )

for file in (dirs):
    #index = file.find('g')     
        
    y_valid[j_valid] = float(labels[labels['name'] == str(file)].iloc[0,2][1:-1])
    if (file[0] == 'l'):
        
        y_valid[j_valid] = -y_valid[j_valid]
    
    #print (y_valid[j_valid,:])
        
    file_picture = "./images_t/valid/" + str(file)
     
    #img = Image.open(file_picture)
    #img.thumbnail((224, 224))
    
    img = load_img(file_picture, target_size=(299, 299))  # this is a PIL image 
    matrix = img_to_array(img)
    X_valid[j_valid,:,:,:] = matrix[:,:,:]
        
    j_valid = j_valid + 1

In [ ]:
j_test = 0

# Open a file
path = "./images_t/te/"
dirs = os.listdir( path )

for file in (dirs):
    #index = file.find('g')     
        
    y_test[j_test] = float(labels[labels['name'] == str(file)].iloc[0,2][1:-1])
    if (file[0] == 'l'):
        
        y_test[j_test] = -y_test[j_test]
    
    #print (y_valid[j_valid,:])
        
    file_picture = "./images_t/tr/" + str(file)
     
    #img = Image.open(file_picture)
    #img.thumbnail((224, 224))
    
    img = load_img(file_picture, target_size=(299, 299))  # this is a PIL image 
    matrix = img_to_array(img)
    X_test[j_test,:,:,:] = matrix[:,:,:]
        
    j_test = j_test + 1

In [6]:
j_train = 0

# Open a file
path = "./images_t/tr/"
dirs = os.listdir( path )

for file in (dirs):
    #index = file.find('g')     
        
    y_train[j_train] = float(labels[labels['name'] == str(file)].iloc[0,2][1:-1])
    if (file[0] == 'l'):
        
        y_train[j_train] = -y_train[j_train]
    
    #print (y_valid[j_valid,:])
        
    file_picture = "./images_t/tr/" + str(file)
     
    #img = Image.open(file_picture)
    #img.thumbnail((224, 224))
    
    img = load_img(file_picture, target_size=(299, 299))  # this is a PIL image 
    matrix = img_to_array(img)
    X_train[j_train,:,:,:] = matrix[:,:,:]
        
    j_train = j_train + 1

In [7]:
# rescaling between 0 and 1 
y_test[:] = 1 + y_test[:]/30
y_train[:] = 1 + y_train[:]/30
y_valid[:] = 1 + y_valid[:]/30

In [8]:
imput_train_Xception = preprocess_input_Xception(X_train)
train_Xception = Xception(weights='imagenet', include_top=False).predict(imput_train_Xception)

print ('done')

imput_test_Xception = preprocess_input_Xception(X_test.copy())
test_Xception = Xception(weights='imagenet', include_top=False).predict(imput_test_Xception)

print ('done')

imput_valid_Xception = preprocess_input_Xception(X_valid)
valid_Xception = Xception(weights='imagenet', include_top=False).predict(imput_valid_Xception)

done
done


In [13]:
def input_branch(input_shape=None):
    
    size = int(input_shape[2] / 4)
    
    branch_input = Input(shape=input_shape)
    branch = GlobalAveragePooling2D()(branch_input)
    branch = Dense(size, use_bias=False, kernel_initializer='uniform')(branch)
    branch = BatchNormalization()(branch)
    branch = Activation("relu")(branch)
    return branch, branch_input

Xception_branch, Xception_input = input_branch(input_shape=(10, 10, 2048))


net = Dropout(0.34)(Xception_branch)
net = Dense(1024, use_bias=False, kernel_initializer='uniform')(net)
net = BatchNormalization()(net)
net = Activation("relu")(net)
net = Dropout(0.34)(net)
net = Dense(1, kernel_initializer='uniform', activation="sigmoid")(net)

model = Model(inputs=[Xception_input], outputs=[net])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 10, 10, 2048)      0         
_________________________________________________________________
global_average_pooling2d_2 ( (None, 2048)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 512)               1048576   
_________________________________________________________________
batch_normalization_15 (Batc (None, 512)               2048      
_________________________________________________________________
activation_3 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 1024)              524288    
__________

In [14]:
# checkpoints

early_stopping = EarlyStopping(monitor='val_acc', patience=5)
STAMP = "{}_dog_breed_model".format(datetime.date.today().strftime("%Y-%m-%d"))

bst_model_path = "./models/{}.h5".format(STAMP)
model_checkpoint = ModelCheckpoint(bst_model_path,
                                   save_best_only=True,
                                   save_weights_only=True)

In [ ]:
model.compile(loss='mean_squared_error', optimizer="sgd", metrics=['mae'])

model.fit([train_Xception],y_train,
          validation_data=([valid_Xception], y_valid),
          epochs=100, batch_size=10,callbacks=[model_checkpoint], verbose=2)

Train on 19012 samples, validate on 5104 samples
Epoch 1/100
 - 27s - loss: 0.2308 - mean_absolute_error: 0.4152 - val_loss: 0.2313 - val_mean_absolute_error: 0.4144
Epoch 2/100
 - 25s - loss: 0.2290 - mean_absolute_error: 0.4133 - val_loss: 0.2315 - val_mean_absolute_error: 0.4139
Epoch 3/100
 - 25s - loss: 0.2276 - mean_absolute_error: 0.4116 - val_loss: 0.2310 - val_mean_absolute_error: 0.4131
Epoch 4/100
 - 25s - loss: 0.2269 - mean_absolute_error: 0.4109 - val_loss: 0.2308 - val_mean_absolute_error: 0.4122
Epoch 5/100
 - 25s - loss: 0.2262 - mean_absolute_error: 0.4097 - val_loss: 0.2316 - val_mean_absolute_error: 0.4108
Epoch 6/100
 - 25s - loss: 0.2248 - mean_absolute_error: 0.4075 - val_loss: 0.2293 - val_mean_absolute_error: 0.4076
Epoch 7/100
 - 25s - loss: 0.2240 - mean_absolute_error: 0.4063 - val_loss: 0.2269 - val_mean_absolute_error: 0.4055
Epoch 8/100
 - 25s - loss: 0.2228 - mean_absolute_error: 0.4043 - val_loss: 0.2247 - val_mean_absolute_error: 0.4016
Epoch 9/100
 - 

In [ ]:
predictions = model.predict([test_Xception])
from sklearn.metrics import mean_absolute_error
print (y_test.shape)
print (predictions.shape)
print (mean_absolute_error(y_test[:], predictions[:]))

In [ ]:
# enregistrer l'angle pour effectuer une rotation 

for index in range (len(predictions)):
    
    liste_angle.append(predictions[index][0])
    
d = {'name': liste_name, 'angle': liste_angle}
df = pd.DataFrame(data=d)
            
df.to_csv('liste_label_predict.csv')

In [ ]:
"""On incline les images (aboutie à la formation de losanges) de l'angle déterminée par l'algorithme anterieur de 
détection automatique d'angles"""

dim = 256

j_te = 0

# Open a file
path = "./images_translat"
dirs = os.listdir( path )

for i in range (4):

    for file in (dirs):
        
        if (str(file)[-4] == '.') or (str(file)[-3] == '.') or (str(file)[-5] == '.'):

            file_picture =  "./images_translat/" + str(file)
            #print (file_picture)   

            img_open = load_img(file_picture, target_size=(dim, dim))  # this is a PIL image  


            j_te = j_te + 1


            img_s = img_to_array(img_open)

            l = img_s.shape[0]
            
            # rescale angle 
            #alpha_deg = np.random.rand(1)*20 + 5
            alpha_deg = -(labels[labels['name'] == ('predict' + str(file))].iloc[0,2] - 0.5)*50
            
            alpha = alpha_deg*np.pi/180
            if (np.abs(alpha_deg)>4):
                

                x = l/(1 + np.absolute(np.sin(alpha)))
                a = x * np.absolute(np.sin(alpha))
                b = x * np.absolute(np.cos(alpha))
                d = a
                c = l - (a+b)   


                pts = np.array([[l,int(np.around(b+c))],[int(np.around(a+c)),l],[int(np.around(c)),int(np.around(l-b))],
                                [int(np.around(l-d)),int(np.around(c))]])

                ## (1) Crop the bounding rect
                rect = cv2.boundingRect(pts)
                x,y,w,h = rect
                croped = img_s[y:y+h, x:x+w].copy()

                ## (2) make mask
                pts = pts - pts.min(axis=0)

                mask = np.zeros(croped.shape[:2], np.uint8)
                cv2.drawContours(mask, [pts], -1, (255, 255, 255), -1, cv2.LINE_AA)

                ## (3) do bit-op
                dst = cv2.bitwise_and(croped, croped, mask=mask)

                ## (4) add the white background
                bg = np.ones_like(croped, np.uint8)*255
                cv2.bitwise_not(bg,bg, mask=mask)
                dst2 = bg+ dst

                mat = np.ones((l,l,3))*255
                mat[int(np.around(c))::,0:-int(np.around(c)),:] = dst2[:,:,:]

                img_end = np.zeros((img_s.shape[0],img_s.shape[1]*2,3))

                img_end[:,:img_end.shape[1]//2,:] = img_s[:,:,:]
                img_end[:,img_end.shape[1]//2:,:] = mat[:,:,:]

                img = array_to_img(img_end)

                if (str(file)[-3:] == 'jpg' ) or (str(file)[-3:] == 'gif' )  or (str(file)[-3:] == 'JPG' ):

                    img.save("./images_translat5/" + str(file)) 

                else :

                    img.save("./images_translat5/" +str(file) ) 